In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


# Install Environment


In [ ]:
!sudo apt-get install python3.8
!sudo apt-get install python3.8-distutils

!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

!git clone https://github.com/mayuelala/FollowYourPose.git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support
  python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support python3.8
  python3.8-minimal
0 upgraded, 6 newly installed, 0 to remove and 18 not upgraded.
Need to get 5,098 kB of archives.
After this operation, 18.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.8-minimal amd64 3.8.18-1+jammy1 [794 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.8-minima

In [ ]:
%cd /content/FollowYourPose
!export PYTHONPATH=/content/FollowYourPose:$PYTHONPATH
!python3.8 -m pip install -q -U --pre triton
!apt update
!python3.8 -m pip install -q diffusers==0.11.1 torch==1.13.1 transformers==4.26.0 bitsandbytes==0.35.4 \
imageio-ffmpeg xformers==0.0.16  --extra-index-url https://download.pytorch.org/whl/cu113
!python3.8 -m pip install -r requirements.txt

/content/FollowYourPose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.2 MB/s eta 0:00:00
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
19 packages can be upgraded. Run 'apt list --upgradable' to see 

# Model

In [ ]:
%mkdir /content/FollowYourPose/checkpoints
%cp -r ../gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints/

# Execution

recommend set video_length=8 in ./config/pose_sample.yaml \
should keep the skeleton frame length(./followyourpose/pipelines/pipeline_followyourpose.py:422 ) equal with video_length (?)

In [ ]:
%cd /content/FollowYourPose
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="configs/pose_sample.yaml"  --skeleton_path="./pose_example/vis_ikun_pose2.mov"

# MOUNT DRIVE 🗻
1. Run the first cell
2. Click on the URL of the Failure in the output
Failure("Error opening URL: ...")
3. Select our ICT3104 shared account
4. Once successful, you can close the tab
5. Run the next cell
6. Done

In [ ]:
# @title First Cell { display-mode: "form" }
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
# @title Next Cell { display-mode: "form" }
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/gdrive/MyDrive

%mkdir /content/video/
%cp -r /content/gdrive/MyDrive/ICT3104/video/* /content/video/

# R2. Exploration

## Steps
1. Login to our shared account
2. Mount the drive (first cell below)
3. Run Video Selection then Setting
    * Reason: Setting cell is observing Video Selection cell

In [ ]:
# @title Video Selection { display-mode: "form" }

import os
import ipywidgets as widgets
from IPython.display import display, HTML, Video, clear_output

selected_video = None

directory_path = '/content/video'
folders = os.listdir(directory_path)
subdirectories = [content for content in folders if os.path.isdir(os.path.join(directory_path, content))]

# Create an output widget as a placeholder
output_placeholder = widgets.Output()
display(output_placeholder)

# Create the subdirectory dropdown
subdir_picker = widgets.Dropdown(
    options=subdirectories,
    value=None,
    description='Folder:',
    disabled=False,
)
display(subdir_picker)

# Create the video dropdown
video_picker = widgets.Dropdown(
    options=[],
    description='Video:',
    disabled=True,
)
display(video_picker)

# Create a button to display the selected video
display_button = widgets.Button(
    description="Display Video",
    disabled=True,
)
button_margin = widgets.Layout(margin='0px 0px 20px 154px')
display_button.layout = button_margin

display(display_button)

# Function to update the video dropdown options based on the selected folder
def update_videos(change):
    selected_folder = change['new']
    directory_path = f"/content/video/{selected_folder}"
    video_options = [file for file in os.listdir(directory_path) if file.endswith('.mp4')]

    video_picker.options = video_options
    video_picker.disabled = False
    if not video_options:
        video_picker.value = None

# Link the subdirectory dropdown to the update function
subdir_picker.observe(update_videos, 'value')

# Function to display the selected video
def display_selected_video(change):
    selected_video = video_picker.value
    if selected_video:
        video_path = f"/content/video/{subdir_picker.value}/{selected_video}"
        video_display = Video(video_path, embed=True)

        # Clear the output placeholder and display the video
        with output_placeholder:
            clear_output()
            display(video_display)

# Link the button to the display function
display_button.on_click(display_selected_video)

# Function to enable the "Display Video" button when a video is selected
def enable_button(change):
    if video_picker.value:
        display_button.disabled = False
    else:
        display_button.disabled = True

# Link the video picker dropdown to the enable button function
video_picker.observe(enable_button, 'value')


Output()

Dropdown(description='Folder:', options=('Charades', 'Samples'), value=None)

Dropdown(description='Video:', disabled=True, options=(), value=None)

Button(description='Display Video', disabled=True, layout=Layout(margin='0px 0px 20px 154px'), style=ButtonSty…

# R3. Inference

In [ ]:
# @title Settings { display-mode: "form" }

from IPython.display import Markdown, display
import ipywidgets as widgets

selected_video_input = widgets.Text(
    placeholder='Select a video above',
    description="Video:",
    disabled=True
)
display(selected_video_input)

text_prompt_input = widgets.Text(
    placeholder='Enter your text prompt here',
    description="Text Prompt:",
    disabled=False,
)
display(text_prompt_input)

print_button = widgets.Button(
    description="Start Inference",
    disabled=True,
)
button_margin = widgets.Layout(margin='0px 0px 20px 154px')
print_button.layout = button_margin
display(print_button)

def update_videos_inf(change):
    video = change['new']
    selected_video_input.value = video

# Link the subdirectory dropdown to the update function
video_picker.observe(update_videos_inf, 'value')

def enable_print_button(change):
    if selected_video_input.value and text_prompt_input.value:
        print_button.disabled = False
    else:
        print_button.disabled = True

# Link the text input widget to the enable_print_button function
text_prompt_input.observe(enable_print_button, 'value')

def print_selections(button):
    selected_video = selected_video_input.value
    text_prompt = text_prompt_input.value

    print(f"Selected Video: {selected_video}")
    print(f"Text Prompt: {text_prompt}")
    print("Imagine Inferencing ✨✨")

# Link the button to the print_selections function
print_button.on_click(print_selections)


Text(value='', description='Video:', disabled=True, placeholder='Select a video above')

Text(value='', description='Text Prompt:', placeholder='Enter your text prompt here')

Button(description='Start Inference', disabled=True, layout=Layout(margin='0px 0px 20px 154px'), style=ButtonS…

# Archive

## Video

In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = "/content/video/subdir2/MWAGL.mp4"
mp4 = open(video_path,'rb').read()

# Needed run this decoder, for some reason collab unable to decode video, and will output as unplayable.
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

# Output
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
# A shorter/simpler code but outputs as unplayable, most prob is source video codec issue
from IPython.display import Video

Video("/content/video/subdir1/E.mp4")

ValueError: ignored